In [12]:
import tensorflow_datasets as tfds
import tensorflow as tf
import keras

In [3]:
dataset, info = tfds.load("tf_flowers", as_supervised = True, with_info = True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

2022-03-28 01:40:54.272952: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 100%|██████████| 5/5 [00:52<00:00, 10.50s/ file]

Dataset tf_flowers downloaded and prepared to /Users/mihirneal/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB




2022-03-28 01:41:59.601255: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-28 01:41:59.602017: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
#Testing the dataset
test_set, valid_set, train_set = tfds.load("tf_flowers", split=['train[20%:]', 'train[:15%]', 'train[:75%]'], as_supervised=True)

In [14]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [16]:
batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [21]:
#Importing the pre-trained Xception Model
base_model = keras.applications.xception.Xception(weights="imagenet", include_top = False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation = "softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs = output)

In [22]:
#Freezing the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [26]:
#Training Model
optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss = "sparse_categorical_crossentropy", optimizer=optimizer, metrics=['Accuracy'])
history = model.fit(train_set, epochs = 10, validation_data=valid_set)

Epoch 1/10


2022-03-28 05:43:08.120070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 1.6824 - Accuracy: 0.7805

2022-03-28 05:43:53.587956: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 55s 591ms/step - loss: 1.6824 - Accuracy: 0.7805 - val_loss: 0.5083 - val_Accuracy: 0.9018
Epoch 2/10
86/86 [==============================] - 50s 582ms/step - loss: 0.5130 - Accuracy: 0.9052 - val_loss: 0.2481 - val_Accuracy: 0.9491
Epoch 3/10
86/86 [==============================] - 50s 577ms/step - loss: 0.2410 - Accuracy: 0.9422 - val_loss: 0.1891 - val_Accuracy: 0.9582
Epoch 4/10
86/86 [==============================] - 50s 581ms/step - loss: 0.1379 - Accuracy: 0.9662 - val_loss: 0.1128 - val_Accuracy: 0.9745
Epoch 5/10
86/86 [==============================] - 51s 589ms/step - loss: 0.0995 - Accuracy: 0.9727 - val_loss: 0.0864 - val_Accuracy: 0.9818
Epoch 6/10
86/86 [==============================] - 50s 581ms/step - loss: 0.0707 - Accuracy: 0.9840 - val_loss: 0.0726 - val_Accuracy: 0.9855
Epoch 7/10
86/86 [==============================] - 49s 576ms/step - loss: 0.0520 - Accuracy: 0.9873 - val_loss: 0.0522 - val_Accuracy: 0.9945
Epoch 8/10

In [27]:
#Using lower learning rate to avoid damaging pre-trained weights
for layer in base_model.layers:
    layer.trainable = True
ptimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)
model.compile(loss = "sparse_categorical_crossentropy", optimizer=optimizer, metrics=['Accuracy'])
history = model.fit(train_set, epochs = 10, validation_data=valid_set)

Epoch 1/10


2022-03-28 05:53:10.652371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 0.4878 - Accuracy: 0.8467

2022-03-28 05:58:03.940380: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 304s 3s/step - loss: 0.4878 - Accuracy: 0.8467 - val_loss: 3.1642 - val_Accuracy: 0.6764
Epoch 2/10
86/86 [==============================] - 230s 3s/step - loss: 0.1405 - Accuracy: 0.9528 - val_loss: 0.0418 - val_Accuracy: 0.9836
Epoch 3/10
86/86 [==============================] - 243s 3s/step - loss: 0.0578 - Accuracy: 0.9811 - val_loss: 0.0273 - val_Accuracy: 0.9927
Epoch 4/10
86/86 [==============================] - 244s 3s/step - loss: 0.0327 - Accuracy: 0.9916 - val_loss: 0.0210 - val_Accuracy: 0.9945
Epoch 5/10
86/86 [==============================] - 235s 3s/step - loss: 0.0142 - Accuracy: 0.9956 - val_loss: 0.0095 - val_Accuracy: 0.9945
Epoch 6/10
86/86 [==============================] - 226s 3s/step - loss: 0.0181 - Accuracy: 0.9942 - val_loss: 0.0021 - val_Accuracy: 1.0000
Epoch 7/10
86/86 [==============================] - 262s 3s/step - loss: 0.0125 - Accuracy: 0.9960 - val_loss: 0.0106 - val_Accuracy: 0.9964
Epoch 8/10
86/86 [======